In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
df=pd.read_excel("./data_raw/crime_rate_2019.xlsx")

In [4]:
df.head()

,State,City,Pop,Violent_Crime_Rate_per_100k,Non_Violent_Crime_Rate_per_100k
0,Alabama,Mobile,248431,6217.02,5453.83
1,Alaska,Anchorage,296188,6640.04,5415.82
2,Arizona,Chandler,249355,2589.08,2329.61
3,Arizona,Gilbert,242090,1483.75,1385.85
4,Arizona,Glendale,249273,5037.85,4530.37


In [5]:
df2 = pd.read_excel("./data_raw/political_affiliation.xlsx", skiprows=1)

In [6]:
df2.head()

,Rank,City,Population (2020),Mayor,Took office,Term ends,Government type,Mayoral election in 2024?
0,1,"New York, New York",8804190,Eric Adams (D),2022.0,2026,Strong mayor,No
1,2,"Los Angeles, California",3898747,Karen Bass (D),2022.0,2026,Strong mayor,No
2,3,"Chicago, Illinois",2746388,Brandon Johnson (D),2023.0,2027,Strong mayor,No
3,4,"Houston, Texas",2304580,John Whitmire (D),2024.0,2028,Strong mayor,No
4,5,"Phoenix, Arizona",1608139,Kate Gallego (D),2019.0,2025,Council-manager,Yes


In [7]:
url = 'https://en.wikipedia.org/wiki/List_of_cities_by_average_temperature#North_America'

In [8]:
response = requests.get(url)

In [9]:
soup = BeautifulSoup(response.content, 'html.parser')

In [10]:
tables = soup.find_all('table')
table_data = []
for table in tables:
    rows = table.find_all('tr')
    for row in rows:
        cells = row.find_all(['th', 'td'])
        table_data.append([cell.get_text(strip=True) for cell in cells])

In [11]:
df3 = pd.DataFrame(table_data)

In [12]:
df3.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Country,City,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year,Ref.
1,Algeria,Algiers,11.2(52.2),11.5(52.7),13.6(56.5),15.6(60.1),18.8(65.8),22.6(72.7),25.7(78.3),26.6(79.9),23.9(75.0),20.4(68.7),15.7(60.3),12.6(54.7),18.2(64.8),[1]
2,Algeria,Tamanrasset,12.8(55.0),15.0(59.0),18.1(64.6),22.2(72.0),26.1(79.0),28.9(84.0),28.7(83.7),28.2(82.8),26.5(79.7),22.4(72.3),17.3(63.1),13.9(57.0),21.7(71.1),[2]
3,Algeria,Reggane,16.0(60.8),18.2(64.8),23.1(73.6),27.9(82.2),32.2(90.0),36.4(97.5),39.8(103.6),38.4(101.1),35.5(95.9),29.2(84.6),22.0(71.6),17.8(64.0),28.3(82.9),[3]
4,Angola,Luanda,26.7(80.1),28.5(83.3),28.6(83.5),28.2(82.8),27.0(80.6),23.9(75.0),22.1(71.8),22.1(71.8),23.5(74.3),25.2(77.4),26.7(80.1),26.9(80.4),25.8(78.4),[4]


In [13]:
df3.columns = df3.iloc[0]
df3 = df3.drop(0)
df3 = df3[df3['Country'] == 'United States']
df3= df3.drop(columns=['Ref.'])

In [14]:
def extract_fahrenheit(s):
    s = s.replace('−', '-')
    fahrenheit_temp = s.split("(")[1].split(")")[0]
    return float(fahrenheit_temp)

In [15]:
for month in df3.columns[2:15]:
    df3[month + '_Fahrenheit'] = df3[month].apply(extract_fahrenheit)

In [16]:
df3.head()

,Country,City,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,...,Apr_Fahrenheit,May_Fahrenheit,Jun_Fahrenheit,Jul_Fahrenheit,Aug_Fahrenheit,Sep_Fahrenheit,Oct_Fahrenheit,Nov_Fahrenheit,Dec_Fahrenheit,Year_Fahrenheit
339,United States,Albuquerque,2.4(36.4),5.2(41.4),8.9(48.1),13.3(56.0),18.7(65.6),23.8(74.9),25.7(78.3),24.6(76.2),...,56.0,65.6,74.9,78.3,76.2,69.3,57.5,44.9,36.3,57.2
340,United States,Anchorage,−8.3(17.1),−6.6(20.2),−3.0(26.6),2.7(36.8),8.8(47.8),12.9(55.2),14.9(58.8),13.7(56.7),...,36.8,47.8,55.2,58.8,56.7,48.6,34.8,22.2,19.0,37.1
341,United States,Atlanta,6.4(43.5),8.6(47.4),12.6(54.6),16.7(62.1),21.3(70.3),25.2(77.4),26.9(80.4),26.4(79.5),...,62.1,70.3,77.4,80.4,79.5,73.5,63.4,54.1,45.5,62.6
342,United States,Austin,10.8(51.5),12.8(55.0),16.5(61.7),20.7(69.2),24.8(76.6),27.9(82.2),29.4(85.0),29.9(85.8),...,69.2,76.6,82.2,85.0,85.8,80.0,71.2,61.0,52.5,69.3
343,United States,Baltimore,0.8(33.5),2.4(36.4),6.8(44.2),12.4(54.3),17.6(63.6),22.8(73.0),25.3(77.6),24.3(75.7),...,54.3,63.6,73.0,77.6,75.7,68.4,56.7,47.0,37.3,55.6


In [18]:
df3.to_csv("./data_raw/temperature.csv", index=False)